In [ ]:
# if the package is not installed in your python environment, run this to execute the notebook directly from inside the GitHub repository
%cd -q ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# some python imports that will be used throughout the tutorial
import numpy as np
import pandas as pd
import pint
import xarray as xr

In [ ]:
# importing the weldx package with prevalent default abbreviations
import weldx
import weldx.geometry as geo
import weldx.transformations as tf
import weldx.utility as ut
import weldx.visualization as vis
from weldx import Q_

### groove shape

In [ ]:
from weldx.welding.groove.iso_9692_1 import get_groove

groove = get_groove(
    groove_type="VGroove",
    workpiece_thickness=Q_(0.5, "cm"),
    groove_angle=Q_(50, "deg"),
    root_face=Q_(1, "mm"),
    root_gap=Q_(1, "mm"),
)

### workpiece geometry

In [ ]:
# define the weld seam length in mm
seam_length = 100

# create a linear trace segment a the complete weld seam trace
trace_segment = geo.LinearHorizontalTraceSegment(seam_length)
trace = geo.Trace(trace_segment)

# create 3d workpiece geometry from the groove profile and trace objects
geometry = geo.Geometry(groove.to_profile(width_default=Q_(50, "mm")), trace)

# rasterize geometry
profile_raster_width = 0.5  # resolution of each profile in mm
trace_raster_width = 15  # space between profiles in mm
geometry_data_sp = geometry.rasterize(
    profile_raster_width=profile_raster_width, trace_raster_width=trace_raster_width
)

## Test rasterization

In [ ]:
geos = geometry.rasterize(
    profile_raster_width=profile_raster_width,
    trace_raster_width=trace_raster_width,
    stack=False,
)

In [ ]:
data, triangle_indices = ut._triangulate_geometry(geos[0])

In [ ]:
geo.PointCloud(*ut._triangulate_geometry(geos[0]))

In [ ]:
ptc_list = [geo.PointCloud.from_geometry_raster(g) for g in geos]

In [ ]:
data, triangle_indices = ut._triangulate_geometry(geos[0])
print(ptc_list[0].coordinates.shape)
print(ptc_list[0].triangles.shape)

## k3d plot

In [ ]:
import k3d

plot = k3d.plot()
for ptc in ptc_list:
    plot += k3d.mesh(ptc.coordinates.flatten().astype("float32"), ptc.triangles.flatten().astype("uint32"))
plot.display()

## ipygany plot

In [ ]:
from ipygany import Component, IsoColor, PolyMesh, Scene, Threshold, Warp


def ipygany_meshes(ptc_list):
    meshes = [PolyMesh(vertices=ptc.coordinates,triangle_indices=ptc.triangles) for ptc in ptc_list]
    return meshes

In [ ]:
meshes = ipygany_meshes(ptc_list)
Scene(meshes)